In [1]:
# %% [code]
import os, glob
import numpy as np
import librosa

INPUT_ROOT = "Wav_data_all"
SPEC_ROOT = r"MFCC_data\Culex\mfcc_raw"
N_MELS     = 128

for wav_path in glob.glob(os.path.join(INPUT_ROOT, "*", "*.wav")):
    species = os.path.basename(os.path.dirname(wav_path))
    out_dir = os.path.join(SPEC_ROOT, species)
    os.makedirs(out_dir, exist_ok=True)

    y, sr = librosa.load(wav_path, sr=None)
    S = librosa.feature.melspectrogram(y, sr=sr, n_mels=N_MELS)
    S_db = librosa.power_to_db(S, ref=np.max)
    fname = os.path.splitext(os.path.basename(wav_path))[0] + ".npy"
    np.save(os.path.join(out_dir, fname), S_db)


In [2]:
# %% [code]
import glob, os
import numpy as np
import tensorflow as tf

# ── Config ─────────────────────────────────────────────
SPEC_ROOT = "MFCC_data/Culex/mfcc_raw"   # where your .npy MFCCs live
IMG_SIZE  = 128                          # resize to this for your CNN
BATCH     = 16                           # training batch size

# 1) Grab all MFCC files
all_paths = glob.glob(os.path.join(SPEC_ROOT, "*.npy"))
print(f"Found {len(all_paths)} files under {SPEC_ROOT}")

# 2) Shuffle & split 80/20
np.random.shuffle(all_paths)
split = int(0.8 * len(all_paths))
train_paths, val_paths = all_paths[:split], all_paths[split:]
print(f"  → {len(train_paths)} train, {len(val_paths)} val")

# 3) Python loader (for tf.py_function)
def load_and_preprocess(path):
    # path is an EagerTensor, so we convert it to a Python string first:
    p = path.numpy().decode("utf-8")
    mfcc = np.load(p)
    # normalize 0…1
    mfcc = (mfcc - mfcc.min()) / (mfcc.max() - mfcc.min() + 1e-6)

    # to TF tensor, add channel dim, resize, and set static shape
    spec = tf.convert_to_tensor(mfcc, tf.float32)
    spec = tf.expand_dims(spec, -1)  # (H, W, 1)
    spec = tf.image.resize(spec, [IMG_SIZE, IMG_SIZE])
    spec.set_shape([IMG_SIZE, IMG_SIZE, 1])

    # single-species => constant label 0
    label = tf.constant(0, dtype=tf.int32)
    label.set_shape([])  # scalar

    return spec, label

# 4) Wrap it for TF graph execution
def tf_parse(path):
    spec, lbl = tf.py_function(
        func=load_and_preprocess,
        inp=[path],
        Tout=(tf.float32, tf.int32)
    )
    # re-set shapes so TF knows them
    spec.set_shape([IMG_SIZE, IMG_SIZE, 1])
    lbl.set_shape([])
    return spec, lbl

# 5) Build tf.data pipelines
def make_ds(paths):
    ds = tf.data.Dataset.from_tensor_slices(paths)
    ds = ds.shuffle(len(paths))
    ds = ds.map(tf_parse, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(BATCH).prefetch(tf.data.AUTOTUNE)
    return ds

train_ds = make_ds(train_paths)
val_ds   = make_ds(val_paths)

# 6) Sanity check
for x_batch, y_batch in train_ds.take(1):
    print("X batch shape:", x_batch.shape)
    print("Y batch shape:", y_batch.shape)


Found 428 files under MFCC_data/Culex/mfcc_raw
  → 342 train, 86 val
X batch shape: (16, 128, 128, 1)
Y batch shape: (16,)


In [3]:

from tensorflow.keras import layers, models
species_list = ["Culex"]
def make_model(input_shape, num_classes):
    inp = layers.Input(input_shape)
    x = layers.Conv2D(32, 3, activation="relu", padding="same")(inp)
    x = layers.MaxPool2D()(x)
    x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
    x = layers.MaxPool2D()(x)
    x = layers.Conv2D(128,3, activation="relu", padding="same")(x)
    x = layers.GlobalAveragePooling2D()(x)
    out = layers.Dense(num_classes, activation="softmax")(x)
    m = models.Model(inp, out)
    m.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
    return m

model = make_model((IMG_SIZE,IMG_SIZE,1), len(species_list))
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 128, 128, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 128)       73856 

In [5]:
# %% [code]
EPOCHS = 100

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)


Epoch 1/100
22/22 [==============================] - 1s 48ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/100
22/22 [==============================] - 1s 34ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/100
 7/22 [========>.....................] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.0000e+00

KeyboardInterrupt: 